In [31]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import pickle

In [32]:
milk_training_data = pd.read_csv('milk_training_data.csv', delimiter=',')

In [33]:
print(milk_training_data.head())

    pH  Temprature  Taste  Odor  Fat   Turbidity  Colour   Grade
0  6.6          35      1     0     1          0     254    high
1  6.6          36      0     1     0          1     253    high
2  8.5          70      1     1     1          1     246     low
3  9.5          34      1     1     0          1     255     low
4  6.6          37      0     0     0          0     255  medium


In [34]:
milk_training_data.rename(columns={'Colour':'Color'}, inplace = True)
milk_training_data.rename(columns={'Temprature':'Temperature'}, inplace = True)
milk_training_data.rename(columns={'Fat ':'Fat'}, inplace = True)

In [35]:
milk_training_data.columns = [col.lower() for col in milk_training_data.columns]

In [36]:
print(milk_training_data.head())

    ph  temperature  taste  odor  fat  turbidity  color   grade
0  6.6           35      1     0    1          0    254    high
1  6.6           36      0     1    0          1    253    high
2  8.5           70      1     1    1          1    246     low
3  9.5           34      1     1    0          1    255     low
4  6.6           37      0     0    0          0    255  medium


In [37]:
k_values = [num for num in range(5, 26, 5)]
print(k_values)

[5, 10, 15, 20, 25]


In [38]:
mean_scores = []

In [39]:
X = milk_training_data.drop('grade', axis=1)
y = milk_training_data['grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

In [40]:
for k in k_values:
    # Создание модели k-ближайших соседей
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    # Вычисление средней оценки точности с использованием перекрестной проверки
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')  # cv=5 означает 5-кратную перекрестную проверку
    mean_scores.append(np.mean(scores))

In [41]:
for k, score in zip(k_values, mean_scores):
    print (f"k ={k} Mean Accuracy: {score}")

k =5 Mean Accuracy: 0.9861635220125787
k =10 Mean Accuracy: 0.9697953984555371
k =15 Mean Accuracy: 0.9483560226096648
k =20 Mean Accuracy: 0.9118223071411512
k =25 Mean Accuracy: 0.9105405620571612


In [42]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)
knnPickle = open('knnpickle_file', 'wb')
pickle.dump(knn, knnPickle)
knnPickle.close()

In [43]:
def forecast_milk_grade(ph, temperature, taste, odor, fat, turbidity, color):
    new_milk_grade = pd.DataFrame({'ph': [ph], 'temperature': [temperature], 'taste': [taste], 'odor': [odor], 'fat': [fat], 'turbidity': [turbidity], 'color': [color]})
    new_milk_grade = new_milk_grade.astype('int')
    new_milk_grade['ph'] = new_milk_grade['ph'].astype('float')
    print(new_milk_grade.info())
    print(new_milk_grade.head())
    loaded_model = pickle.load(open('knnpickle_file', 'rb'))
    forecast = knn.predict(new_milk_grade)
    return print(f"Прогноз качества молока: {forecast}")

forecast_milk_grade(6.6, 35, 0, 0, 0, 0, 254)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ph           1 non-null      float64
 1   temperature  1 non-null      int32  
 2   taste        1 non-null      int32  
 3   odor         1 non-null      int32  
 4   fat          1 non-null      int32  
 5   turbidity    1 non-null      int32  
 6   color        1 non-null      int32  
dtypes: float64(1), int32(6)
memory usage: 160.0 bytes
None
    ph  temperature  taste  odor  fat  turbidity  color
0  6.0           35      0     0    0          0    254
Прогноз качества молока: ['medium']
